In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
np.random.seed(73298)

In [3]:
generated_dataset = pickle.load(open("first_dataset.pickle", "rb"))

INFO:helper:Reading bedfile...
INFO:helper:Done reading bedfile ../indexes/yeast-all.bed
INFO:helper:Processed TIFs in chromosome 1...
INFO:helper:Processed TIFs in chromosome 2...
INFO:helper:Processed TIFs in chromosome 3...
INFO:helper:Processed TIFs in chromosome 4...
INFO:helper:Processed TIFs in chromosome 5...
INFO:helper:Processed TIFs in chromosome 6...
INFO:helper:Processed TIFs in chromosome 7...
INFO:helper:Processed TIFs in chromosome 8...
INFO:helper:Processed TIFs in chromosome 9...
INFO:helper:Processed TIFs in chromosome 10...
INFO:helper:Processed TIFs in chromosome 11...
INFO:helper:Processed TIFs in chromosome 12...
INFO:helper:Processed TIFs in chromosome 13...
INFO:helper:Processed TIFs in chromosome 14...
INFO:helper:Processed TIFs in chromosome 15...
INFO:helper:Processed TIFs in chromosome 16...
INFO:helper:Done reading TIF file... returning a dictionary
INFO:helper:Reading bedfile...
INFO:helper:Done reading bedfile ../indexes/yeast-all.bed
INFO:helper:Reading

In [4]:
generated_dataset[0]

{'chrom': 'chrI',
 'strand': '+',
 'gene': 'YAL066W',
 'isoforms': [<simulate_reads.Isoform at 0x7ff554130320>,
 'distr': array([0.25600663, 0.13504557, 0.6089478 ]),
 'density': [2,
  6,
  8,
  13,
  16,
  18,
  21,
  24,
  25,
  27,
  33,
  35,
  37,
  39,
  40,
  44,
  50,
  53,
  56,
  59,
  63,
  72,
  73,
  78,
  82,
  85,
  90,
  95,
  100,
  102,
  103,
  105,
  107,
  109,
  117,
  120,
  123,
  127,
  130,
  132,
  133,
  135,
  136,
  140,
  141,
  143,
  140,
  141,
  140,
  140,
  144,
  142,
  142,
  148,
  153,
  149,
  149,
  149,
  150,
  151,
  150,
  146,
  151,
  150,
  153,
  154,
  147,
  147,
  145,
  143,
  145,
  144,
  143,
  141,
  141,
  144,
  147,
  148,
  147,
  140,
  140,
  141,
  141,
  141,
  144,
  150,
  152,
  152,
  151,
  151,
  153,
  156,
  156,
  153,
  153,
  150,
  153,
  155,
  152,
  149,
  149,
  148,
  150,
  153,
  154,
  153,
  154,
  148,
  150,
  146,
  142,
  145,
  148,
  149,
  150,
  152,
  149,
  150,
  150,
  153,
  150,
  149,

In [5]:
import helper
genome = helper.organize_genome_by_chrom(helper.read_bedfile())

INFO:helper:Reading bedfile...
INFO:helper:Done reading bedfile ../indexes/yeast-all.bed


In [103]:
window = 100
colnames = [str(i) for i in range(-100, 101)] + ['Isoform']

labeled_data = []

keep_non_isoforms_threshold = 0.01

In [104]:
for entry in generated_dataset: 
    isoform_starts = [x.start for x in entry['isoforms']]
    isoform_starts = sorted(isoform_starts)
    gene_start = isoform_starts[0]
    utr_end = int(genome[entry['chrom']][entry['gene']].fields[6])
    utr_end = min(utr_end, isoform_starts[-1])
    for pos in range(gene_start + window, utr_end - window + 1):
        isoform =  int(pos + window in isoform_starts)
        if isoform == 0:
            if np.random.uniform() > keep_non_isoforms_threshold:
                continue
        ws = (pos-window) - gene_start # density array is indexed from 0. pos 0 is gene start
        we = (pos+window+1) - gene_start 
        row = entry['density'][ws:we] + [isoform]
    
        labeled_data.append(dict(zip(colnames, row)))

In [105]:
labeled_data = pd.DataFrame(labeled_data)
labeled_data

,-100,-99,-98,-97,-96,-95,-94,-93,-92,-91,...,92,93,94,95,96,97,98,99,100,Isoform
0,16,18,21,24,25,27,33,35,37,39,...,168,168,174,172,170,167,171,170,170,1
1,107,109,117,120,123,127,130,132,133,135,...,207,205,202,202,204,203,203,204,212,1
2,146,149,154,158,164,167,170,172,172,168,...,188,187,187,189,188,186,186,191,197,0
3,186,184,183,184,179,175,176,179,176,176,...,182,183,180,177,177,179,176,178,177,0
4,176,178,174,168,164,158,155,157,158,155,...,181,180,180,179,183,184,178,178,182,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,7,7,7,7,7,6,6,6,6,6,...,9,9,9,9,9,10,9,9,10,1
2216,6,6,6,6,6,6,6,6,6,6,...,9,8,8,8,8,8,9,9,10,1
2217,6,6,6,6,5,5,5,4,4,5,...,10,10,10,10,10,12,12,11,12,1
2218,4,4,5,6,7,7,7,7,7,7,...,15,16,17,18,19,19,19,20,21,1


In [106]:
labeled_data['Isoform'].value_counts()

1    1448
0     772
Name: Isoform, dtype: int64

In [107]:
from sklearn.model_selection import train_test_split

In [108]:
y = labeled_data.pop('Isoform')
X = labeled_data

In [109]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [89]:
from keras.models import Sequential
from keras.layers import Dense

In [110]:
classifier = Sequential()
classifier.add(Dense(units = 201, activation='sigmoid', input_dim=201))
classifier.add(Dense(units = 402, activation='relu'))

classifier.add(Dense(units = 1, activation='sigmoid'))
classifier.compile(optimizer='sgd', loss='binary_crossentropy')



In [111]:
classifier.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6590
Epoch 2/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6466
Epoch 3/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6430
Epoch 4/10
1332/1332 [==============================] - 3s 3ms/step - loss: 0.6406
Epoch 5/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6423
Epoch 6/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6394
Epoch 7/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6427
Epoch 8/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6392
Epoch 9/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6403
Epoch 10/10
1332/1332 [==============================] - 3s 2ms/step - loss: 0.6388


In [112]:
y_pred = classifier.predict(X_test)
y_pred = [1 if Y >= 0.5 else 0 for Y in y_pred]


In [113]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6463963963963963

In [114]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  1, 314],
       [  0, 573]])